In [1]:
import numpy as np

In [2]:
##function for dataset creation
def data():
    inputs = np.random.randint(0, 9, 10)
    targets = np.sort(inputs)
    return inputs, targets

In [3]:
inputs, targets = data()

In [4]:
inputs

array([4, 3, 2, 2, 0, 6, 0, 2, 1, 6])

In [5]:
targets

array([0, 0, 1, 2, 2, 2, 3, 4, 6, 6])

In [6]:
###ENCODER
class RNN:
    def __init__(self):
        
        self.hidden_layers = 100
        self.seq_length = 10
        self.learning_rate = 1e-1
        self.unique_chars = 10
        self.hp = np.zeros((100, 1))
        
        
        self.Wxh = np.random.randn(self.hidden_layers, self.unique_chars) * 0.01 ###input weight(between the input and the hidden state)
        self.Whh = np.random.randn(self.hidden_layers, self.hidden_layers) * 0.01 ###hidden state weight(between the previous hidden state and current hidden state)
        self.Why = np.random.randn(self.unique_chars, self.hidden_layers) * 0.01 ###output weight(between the hidden state and output)

        self.bh = np.zeros((self.hidden_layers, 1))  ##hidden state bias
        self.by = np.zeros((self.unique_chars, 1))   ##output state bias
        
        
        
        
    def sample(self, h, seed_ix, n):
        x = np.zeros((self.unique_chars, 1))
        x[seed_ix] = 1
        ixes = []

        for t in range(n):
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) +self.bh)
            y = np.dot(self.Why, h) + self.by
            p = np.exp(y) / np.sum(np.exp(y))
            ix = np.random.choice(range(self.unique_chars), p=p.ravel())
            x = np.zeros((self.unique_chars, 1))
            x[ix] = 1
            ixes.append(ix)

        print (ixes)

    
    
    def __init__(self):
        
        self.hidden_layers = 100
        self.seq_length = 10
        self.learning_rate = 1e-1
        self.unique_chars = 10
        self.hp = np.zeros((100, 1))
        
        
        self.Wxh = np.random.randn(self.hidden_layers, self.unique_chars) * 0.01 ###input weight(between the input and the hidden state)
        self.Whh = np.random.randn(self.hidden_layers, self.hidden_layers) * 0.01 ###hidden state weight(between the previous hidden state and current hidden state)
        self.Why = np.random.randn(self.unique_chars, self.hidden_layers) * 0.01 ###output weight(between the hidden state and output)

        self.bh = np.zeros((self.hidden_layers, 1))  ##hidden state bias
        self.by = np.zeros((self.unique_chars, 1))   ##output state bias
        
        
        
        
########DECODER            
    def loss_fxn(self, inputs, targets, hp):
        xs, hs, ys, ps = {}, {}, {}, {}


        hs[-1] = np.copy(self.hp)
        loss = 0    ##initialize loss as zero

        ##forward propagation
        for t in range(len(inputs)):
            xs[t]  = np.zeros((self.unique_chars, 1))
            xs[t][inputs[t]] = 1
            hs[t] = np.tanh(np.dot(self.Wxh, xs[t]) + np.dot(self.Whh, hs[t-1]) + self.bh)
            ys[t] = np.dot(self.Why, hs[t]) + self.by
            ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
            loss += -np.log(ps[t][targets[t], 0]) ##cross-entropy


        ###backward propagation
        dWxh, dWhh, dWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)
        dhnext = np.zeros_like(hs[0])

        for t in reversed(range(len(inputs))):
            dy = np.copy(ps[t])
            dy[targets[t]] -= 1
            dWhy += np.dot(dy, hs[t].T)
            dby += dy

            dh = np.dot(self.Why.T, dy) + dhnext
            dhraw = (1 - hs[t] * hs[t]) * dh
            dbh += dhraw
            dWxh += np.dot(dhraw, xs[t].T)
            dWhh += np.dot(dhraw, hs[t-1].T)
            dhnext = np.dot(self.Whh.T, dhraw) 

        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -5, 5, out=dparam)
        return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]
    
    
    def train(self):
        
        n, p = 0, 0
        
        mWxh, mWhh, mWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        mbh, mby = np.zeros_like(self.bh), np.zeros_like(self.by) # memory variables for Adagrad                                                                                                                
        smooth_loss = -np.log(1.0/self.unique_chars)*self.seq_length # loss at iteration 0                                                                                                                        
        while n<=1000*10:

                                                                                                                                
            loss, dWxh, dWhh, dWhy, dbh, dby, hp = self.loss_fxn(inputs, targets, self.hp)
            smooth_loss = smooth_loss * 0.999 + loss * 0.001

              # sample from the Encoder model now and then                                                                                                                                                        
            if n % 1000 == 0:
                print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
                
                self.sample(self.hp, inputs[0], len(targets))

              # perform parameter update with Adagrad                                                                                                                                                     
            for param, dparam, mem in zip([self.Wxh, self.Whh, self.Why, self.bh, self.by],
                                            [dWxh, dWhh, dWhy, dbh, dby],
                                            [mWxh, mWhh, mWhy, mbh, mby]):
                mem += dparam * dparam
                param += -self.learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

            p += self.seq_length # move data pointer                                                                                                                                                         
            n += 1 # iteration counter

        
        

        
        
        
        
    

In [7]:
rnn = RNN()

In [8]:
hp = np.zeros((100, 1))

In [10]:
#rnn.loss_fxn(inputs, targets, hp)

In [11]:
rnn.train()

iter 0, loss: 23.025848
[9, 1, 7, 1, 9, 2, 7, 1, 2, 9]
iter 1000, loss: 8.686192
[0, 0, 1, 2, 2, 2, 3, 4, 6, 6]
iter 2000, loss: 3.197498
[0, 0, 1, 2, 2, 2, 3, 4, 6, 6]
iter 3000, loss: 1.177809
[0, 0, 1, 2, 2, 2, 3, 4, 6, 6]
iter 4000, loss: 0.434554
[0, 0, 1, 2, 2, 2, 3, 4, 6, 6]
iter 5000, loss: 0.160895
[0, 0, 1, 2, 2, 2, 3, 4, 6, 6]
iter 6000, loss: 0.060054
[0, 0, 1, 2, 2, 2, 3, 4, 6, 6]
iter 7000, loss: 0.022835
[0, 0, 1, 2, 2, 2, 3, 4, 6, 6]
iter 8000, loss: 0.009049
[0, 0, 1, 2, 2, 2, 3, 4, 6, 6]
iter 9000, loss: 0.003903
[0, 0, 1, 2, 2, 2, 3, 4, 6, 6]
iter 10000, loss: 0.001951
[0, 0, 1, 2, 2, 2, 3, 4, 6, 6]


In [12]:
inputs

array([4, 3, 2, 2, 0, 6, 0, 2, 1, 6])

In [13]:
targets

array([0, 0, 1, 2, 2, 2, 3, 4, 6, 6])